In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import csv
import os
from os.path import isfile, join
import re
import h5py
import random

# Create Datasets EA and RE

# Cogemos grupos consecutivos de 64 frames y de esos 64 cogemos los multiplos de 8 (1,8,16,24,32,40,48 y 56)

In [3]:
def createDataset_RE(Dataset_rat):
    
    Y_list = [] #Lista en la que vamos a almacenar los valores de Y

    dataset = h5py.File(Dataset_rat,'r')


    #Recorremos el dataset, si la posicion 0 es una Y, añadimos el valor a nuestra lista
    for line in dataset:
        if line[0] == 'Y':
            Y_list.append(line)


    Y_list.sort(key = len) #Nos ordena la lista. Y va acompañado de un valor numerico pero al estar en forma de string, 
                           #tenemos que ordenarlos en funcion de su longitud

    allRE = [] #Lista que almacenara todos los valores de Y con un estado RE
    count = 0 #Contador para comprobar si son 64 frames consecutivos con el mismo estado RE

    for i in Y_list:  #Recorremos la lista y las lineas del dataset
        for j in dataset:

            if i == j: #Si coinciden los valores, comprobamos que el estado de ese valor sea RE
                if dataset[j].value == 'RE': #si el estado es RE lo añadimos a la lista y sumamos 1 al contador
                    count += 1
                    allRE.append(j + '/' +str(count)) #ademas tambien le vamos a añadir al nombre el contador para poder
                                                      #analizar posteriormente si son consecutivos 

                    if count == 65: #cuando el contador llegue a 65 (empieza en 1), se volvera a poner a 0
                        count = 0

                if dataset[j].value == 'EA': #si encuentra un valor de EA para Y, se pondra a 0 el contador
                    count = 0
                    
    sixfourRE = [] #en esta lista meteremos todos los valores solo si son consecutivos 64 veces.
    a = False

    for h in reversed(allRE): #leemos la lista de forma inversa, para comprobar que son consecutivos haremos una cuenta atras
                              # si empieza en 65 empezara a añadir valores a la lista
        b = h.find('/')

        if h[b:] == '/65': 
            a = True

        if a == True:
            sixfourRE.append(h)

        if h[b:] == '/1': #Cada vez que el numero llegue a 1 el estado del booleano cambiara a false y dejara de agregar a la
                          # lista datos, hasta que vuelva a encontrar un 65
            a = False    
            
            
    group8RE = [] #De cada grupo de 64, cogeremos 8 (los que sean multiplos de 8) y los guardaremos en esta lista

    for line in reversed(sixfourRE): #recorremos la lista anterior invertida (la volvemos a poner en orden ascendente)
        b = line.find('/')
        if (line[b:] == '/1' or line[b:] == '/9' or line[b:] == '/17' or line[b:] == '/25' or line[b:] == '/33' 
        or line[b:] == '/41' or line[b:] == '/49' or line[b:] == '/57'): # si es multiplo de 8 (le sumamos 1, por que no
                                                                         # empieza desde 0), lo añadimos a la nueva lista
            group8RE.append(line[:b])    
            
    ##########################################################################################################################
    # Sabiendo ya los estados, vamos a sacar sus frames correspondientes
    ##########################################################################################################################

    it = 1 #Creamos un contador que nos indicara en que frame nos encontramos
    
    name_video = Dataset_rat[37:]
    
    cap = cv2.VideoCapture('/mnt/MD1200A/lconcha/videos/RAT_VIDEOS_CROPPED_RESIZE/'+ name_video +'.mp4') #archivo de video que vamos a leer

    np_frames = [] #lista en la que vamos a guardar los frames 

    while (cap.isOpened()): #leemos el video al cual vamos a crear su dataset

        ret, frame = cap.read()

        if ret == True:
            
            resize = cv2.resize(frame, (128,128), interpolation = cv2.INTER_AREA)
            for line in group8RE: #loope que recorre la lista con los grupos de 8
                if line[1:] == str(it): #si el numero coincide con el frame, lo añadimos a nuestra lista
                    np_frames.append(resize)
            it += 1

        if ret == False:
            break
            
    sets = [] #En esta lista vamos a separar en conjuntos de 8 los frames obtenidos anteriormente
    X_values = [] #En esta lista cada conjunto de 8 frames lo convertiremos en un array de shape [8 512 512]
    state = 'RE' #estado de todos los frames

    it = 0

    for i in range(0, len(np_frames), 8): #Recorremos np_frames, y los añadimos de 8 en 8 a sets
        sets.append(np_frames[i:i+8])

    for j in sets: #recorremos sets y concatenamos los 8 frames, para añadirlos a una nueva lista
        X = np.stack(j)
        X_values.append(X)

    with h5py.File('/mnt/MD1200A/lconcha/videos/Datasets_RE/'+ name_video + '_RE', 'w') as h5File: #Creamos en archivo .h5
        for line in X_values:
            it += 1
            Xset = h5File.create_dataset('X'+str(it), data= line)
            Yset = h5File.create_dataset('Y'+str(it), data = state)

In [4]:
def createDataset_EA(Dataset_rat):

    ##########################################################################################################################
    # Ahora creamos un dataset para el estado de ataque, hacemos lo mismo pero con los frames de EA
    ##########################################################################################################################

    Y_list = [] #Lista en la que vamos a almacenar los valores de Y

    dataset = h5py.File(Dataset_rat,'r')

    for line in dataset:
        if line[0] == 'Y':
            Y_list.append(line)

    Y_list.sort(key = len)

    allEA = [] #Lista que almacenara todos los valores de Y con un estado EA
    count = 0 

    for i in Y_list:
        for j in dataset:

            if i == j: 
                if dataset[j].value == 'EA': 
                    count += 1
                    allEA.append(j + '/' +str(count))  

                    if count == 65: 
                        count = 0

                if dataset[j].value == 'RE':
                    count = 0

    sixfourEA = [] #en esta lista meteremos todos los valores solo si son consecutivos 64 veces.
    a = False

    for h in reversed(allEA): 
        b = h.find('/')

        if h[b:] == '/65': 
            a = True

        if a == True:
            sixfourEA.append(h)

        if h[b:] == '/1': #Cada vez que el numero llegue a 1 el estado del booleano cambiara a false y dejara de agregar a la
                          # lista datos, hasta que vuelva a encontrar un 65
            a = False

    group8EA = [] #De cada grupo de 64, cogeremos 8 (los que sean multiplos de 8) y los guardaremos en esta lista

    for line in reversed(sixfourEA): 
        b = line.find('/')
        if (line[b:] == '/1' or line[b:] == '/9' or line[b:] == '/17' or line[b:] == '/25' or line[b:] == '/33' 
        or line[b:] == '/41' or line[b:] == '/49' or line[b:] == '/57'): 
            group8EA.append(line[:b])

    ##########################################################################################################################
    # Sabiendo ya los estados, vamos a sacar sus frames correspondientes
    ##########################################################################################################################

    it = 1 

    name_video = Dataset_rat[37:] 

    cap = cv2.VideoCapture('/mnt/MD1200A/lconcha/videos/RAT_VIDEOS_CROPPED_RESIZE/'+ name_video +'.mp4') #archivo de video que vamos a leer 

    np_frames_EA = [] 

    while (cap.isOpened()): 

        ret, frame = cap.read()

        if ret == True:
            
            resize = cv2.resize(frame, (128,128), interpolation = cv2.INTER_AREA)
            for line in group8EA: 
                if line[1:] == str(it): 
                    np_frames_EA.append(resize)
            it += 1

        if ret == False:
            break

    sets = [] 
    X_values = [] 
    state = 'EA' 

    it = 0

    for i in range(0, len(np_frames_EA), 8): 
        sets.append(np_frames_EA[i:i+8])

    for j in sets: 
        X = np.stack(j)
        X_values.append(X)

    with h5py.File('/mnt/MD1200A/lconcha/videos/Datasets_EA/'+ name_video + '_EA', 'w') as h5File: #Creamos en archivo .h5
        for line in X_values:
            it += 1
            Xset = h5File.create_dataset('X'+str(it), data= line)
            Yset = h5File.create_dataset('Y'+str(it), data = state)

In [ ]:
#Le decimos la ruta 
ruta = r'/mnt/MD1200A/lconcha/videos/Datasets'

#hacemos una lista de todos los archivos de la ruta
archivo = os.listdir(ruta)

#un loop para que ejecute el programa anterior a todos los elementos de la lista
for i in archivo:
    createDataset_EA('/mnt/MD1200A/lconcha/videos/Datasets/' + i)
    print (i + ' '+ 'completed_EA')
    createDataset_RE('/mnt/MD1200A/lconcha/videos/Datasets/' + i)
    print (i + ' '+ 'completed_RE')

/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:31: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


1.2018-1-9.Rat8 completed_EA


/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:32: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


1.2018-1-9.Rat8 completed_RE
1.2018-1-9.Rat9 completed_EA
1.2018-1-9.Rat9 completed_RE
10.2017-11-10.Rat11 completed_EA
10.2017-11-10.Rat11 completed_RE
10.2017-11-10.Rat12 completed_EA
10.2017-11-10.Rat12 completed_RE
11.2017-11-13.Rat11 completed_EA
11.2017-11-13.Rat11 completed_RE
11.2017-11-13.Rat12 completed_EA
11.2017-11-13.Rat12 completed_RE
12.2017-12-13.Rat13 completed_EA
12.2017-12-13.Rat13 completed_RE
12.2017-12-13.Rat18 completed_EA
12.2017-12-13.Rat18 completed_RE
13.2017-12-14.Rat13 completed_EA
13.2017-12-14.Rat13 completed_RE
13.2017-12-14.Rat18 completed_EA
13.2017-12-14.Rat18 completed_RE
14.2017-12-14.Rat13 completed_EA
14.2017-12-14.Rat13 completed_RE
14.2017-12-14.Rat18 completed_EA
14.2017-12-14.Rat18 completed_RE
15.2017-12-15.Rat13 completed_EA
15.2017-12-15.Rat13 completed_RE
15.2017-12-15.Rat18 completed_EA
15.2017-12-15.Rat18 completed_RE
16.2018-1-19.Rat13 completed_EA
16.2018-1-19.Rat13 completed_RE
16.2018-1-19.Rat18 completed_EA
16.2018-1-19.Rat18 comple

60.2018-1-7.Rat27 completed_RE
60.2018-1-7.Rat30 completed_EA
60.2018-1-7.Rat30 completed_RE
61.2018-1-7.Rat26 completed_EA
61.2018-1-7.Rat26 completed_RE
61.2018-1-7.Rat27 completed_EA
61.2018-1-7.Rat27 completed_RE
61.2018-1-7.Rat30 completed_EA
61.2018-1-7.Rat30 completed_RE
62.2018-1-7.Rat26 completed_EA
62.2018-1-7.Rat26 completed_RE
62.2018-1-7.Rat27 completed_EA
62.2018-1-7.Rat27 completed_RE
62.2018-1-7.Rat30 completed_EA
62.2018-1-7.Rat30 completed_RE
63.2018-2-13.Rat25 completed_EA
63.2018-2-13.Rat25 completed_RE
63.2018-2-13.Rat26 completed_EA
63.2018-2-13.Rat26 completed_RE
63.2018-2-13.Rat30 completed_EA
63.2018-2-13.Rat30 completed_RE
64.2018-2-13.Rat25 completed_EA
64.2018-2-13.Rat25 completed_RE
64.2018-2-13.Rat26 completed_EA
64.2018-2-13.Rat26 completed_RE
64.2018-2-13.Rat30 completed_EA
64.2018-2-13.Rat30 completed_RE
65.2018-2-13.Rat25 completed_EA
65.2018-2-13.Rat25 completed_RE
65.2018-2-13.Rat26 completed_EA
65.2018-2-13.Rat26 completed_RE
65.2018-2-13.Rat30 comp

138.2018-2-6.Rat36 completed_RE
139.2018-2-6.Rat34 completed_EA
139.2018-2-6.Rat34 completed_RE
139.2018-2-6.Rat36 completed_EA
139.2018-2-6.Rat36 completed_RE
140.2018-2-6.Rat34 completed_EA
140.2018-2-6.Rat34 completed_RE
140.2018-2-6.Rat36 completed_EA
140.2018-2-6.Rat36 completed_RE
141.2018-2-6.Rat34 completed_EA
141.2018-2-6.Rat34 completed_RE
141.2018-2-6.Rat36 completed_EA
141.2018-2-6.Rat36 completed_RE
142.2018-2-6.Rat34 completed_EA
142.2018-2-6.Rat34 completed_RE
142.2018-2-6.Rat36 completed_EA
142.2018-2-6.Rat36 completed_RE
143.2018-2-7.Rat34 completed_EA
143.2018-2-7.Rat34 completed_RE
143.2018-2-7.Rat36 completed_EA
143.2018-2-7.Rat36 completed_RE
144.2018-2-7.Rat34 completed_EA
144.2018-2-7.Rat34 completed_RE
144.2018-2-7.Rat36 completed_EA
144.2018-2-7.Rat36 completed_RE
145.2018-2-7.Rat34 completed_EA
145.2018-2-7.Rat34 completed_RE
145.2018-2-7.Rat36 completed_EA
145.2018-2-7.Rat36 completed_RE
146.2018-2-7.Rat34 completed_EA
146.2018-2-7.Rat34 completed_RE
146.2018

179.2018-5-17.Rat52 completed_EA
179.2018-5-17.Rat52 completed_RE
179.2018-5-17.Rat54 completed_EA
179.2018-5-17.Rat54 completed_RE
179.2018-5-17.Rat55 completed_EA
179.2018-5-17.Rat55 completed_RE
179.2018-5-17.Rat56 completed_EA
179.2018-5-17.Rat56 completed_RE
179.2018-5-17.Rat58 completed_EA
179.2018-5-17.Rat58 completed_RE
180.2018-5-17.Rat52 completed_EA
180.2018-5-17.Rat52 completed_RE
180.2018-5-17.Rat54 completed_EA
180.2018-5-17.Rat54 completed_RE
180.2018-5-17.Rat55 completed_EA
180.2018-5-17.Rat55 completed_RE
180.2018-5-17.Rat56 completed_EA
180.2018-5-17.Rat56 completed_RE
180.2018-5-17.Rat58 completed_EA
180.2018-5-17.Rat58 completed_RE
181.2018-5-17.Rat52 completed_EA
181.2018-5-17.Rat52 completed_RE
181.2018-5-17.Rat54 completed_EA
181.2018-5-17.Rat54 completed_RE
181.2018-5-17.Rat55 completed_EA
181.2018-5-17.Rat55 completed_RE
181.2018-5-17.Rat56 completed_EA
181.2018-5-17.Rat56 completed_RE
181.2018-5-17.Rat58 completed_EA
181.2018-5-17.Rat58 completed_RE
182.2018-5

In [52]:
#Le decimos la ruta 
ruta1 = r'/mnt/MD1200A/lconcha/videos/Datasets'
ruta2 = r'/mnt/MD1200A/lconcha/videos/Datasets_EA/'

#hacemos una lista de todos los archivos de la ruta
archivo1 = os.listdir(ruta1)
archivo2 = os.listdir(ruta2)
archivo3 = []

for l in archivo2:
    name = l.find('_')
    archivo3.append(l[:name])
       
        

lista = list(set(archivo1)-set(archivo3))


for i in lista:
    createDataset_EA('/mnt/MD1200A/lconcha/videos/Datasets/' + i)
    print (i + ' '+ 'completed_EA')
    createDataset_RE('/mnt/MD1200A/lconcha/videos/Datasets/' + i)
    print (i + ' '+ 'completed_RE')   
        

            

/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:31: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


81.2018-2-14.Rat25 completed_EA


/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:32: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


81.2018-2-14.Rat25 completed_RE
90.2018-2-14.Rat26 completed_EA
90.2018-2-14.Rat26 completed_RE
88.2018-2-14.Rat30 completed_EA
88.2018-2-14.Rat30 completed_RE
82.2018-2-14.Rat26 completed_EA
82.2018-2-14.Rat26 completed_RE
87.2018-2-14.Rat26 completed_EA
87.2018-2-14.Rat26 completed_RE
86.2018-2-14.Rat26 completed_EA
86.2018-2-14.Rat26 completed_RE
82.2018-2-14.Rat25 completed_EA
82.2018-2-14.Rat25 completed_RE
85.2018-2-14.Rat25 completed_EA
85.2018-2-14.Rat25 completed_RE
83.2018-2-14.Rat26 completed_EA
83.2018-2-14.Rat26 completed_RE
86.2018-2-14.Rat25 completed_EA
86.2018-2-14.Rat25 completed_RE
83.2018-2-14.Rat30 completed_EA
83.2018-2-14.Rat30 completed_RE
84.2018-2-14.Rat26 completed_EA
84.2018-2-14.Rat26 completed_RE
87.2018-2-14.Rat30 completed_EA
87.2018-2-14.Rat30 completed_RE
79.2018-2-14.Rat30 completed_EA
79.2018-2-14.Rat30 completed_RE
87.2018-2-14.Rat25 completed_EA
87.2018-2-14.Rat25 completed_RE
82.2018-2-14.Rat30 completed_EA
82.2018-2-14.Rat30 completed_RE
84.2018-

In [6]:
createDataset_RE('/mnt/MD1200A/lconcha/videos/Datasets/4.2017-11-10.Rat12')

/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:32: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


# # Cogemos grupos consecutivos de 32 frames y de esos 32 cogemos los multiplos de 4 (1,4,8,12,16,20,24 y 28)

In [7]:
def createDataset_RE_group32(Dataset_rat):
    
    Y_list = [] #Lista en la que vamos a almacenar los valores de Y

    dataset = h5py.File(Dataset_rat,'r')


    #Recorremos el dataset, si la posicion 0 es una Y, añadimos el valor a nuestra lista
    for line in dataset:
        if line[0] == 'Y':
            Y_list.append(line)


    Y_list.sort(key = len) #Nos ordena la lista. Y va acompañado de un valor numerico pero al estar en forma de string, 
                           #tenemos que ordenarlos en funcion de su longitud

    allRE = [] #Lista que almacenara todos los valores de Y con un estado RE
    count = 0 #Contador para comprobar si son 32 frames consecutivos con el mismo estado RE

    for i in Y_list:  #Recorremos la lista y las lineas del dataset
        for j in dataset:

            if i == j: #Si coinciden los valores, comprobamos que el estado de ese valor sea RE
                if dataset[j].value == 'RE': #si el estado es RE lo añadimos a la lista y sumamos 1 al contador
                    count += 1
                    allRE.append(j + '/' +str(count)) #ademas tambien le vamos a añadir al nombre el contador para poder
                                                      #analizar posteriormente si son consecutivos 

                    if count == 33: #cuando el contador llegue a 33 (empieza en 1), se volvera a poner a 0
                        count = 0

                if dataset[j].value == 'EA': #si encuentra un valor de EA para Y, se pondra a 0 el contador
                    count = 0
                    
    sixfourRE = [] #en esta lista meteremos todos los valores solo si son consecutivos 32 veces.
    a = False

    for h in reversed(allRE): #leemos la lista de forma inversa, para comprobar que son consecutivos haremos una cuenta atras
                              # si empieza en 33 empezara a añadir valores a la lista
        b = h.find('/')

        if h[b:] == '/33': 
            a = True

        if a == True:
            sixfourRE.append(h)

        if h[b:] == '/1': #Cada vez que el numero llegue a 1 el estado del booleano cambiara a false y dejara de agregar a la
                          # lista datos, hasta que vuelva a encontrar un 33
            a = False    
            
            
    group8RE = [] #De cada grupo de 32, cogeremos 8 (los que sean multiplos de 4) y los guardaremos en esta lista

    for line in reversed(sixfourRE): #recorremos la lista anterior invertida (la volvemos a poner en orden ascendente)
        b = line.find('/')
        if (line[b:] == '/1' or line[b:] == '/5' or line[b:] == '/9' or line[b:] == '/13' or line[b:] == '/17' 
        or line[b:] == '/21' or line[b:] == '/25' or line[b:] == '/29'): # si es multiplo de 8 (le sumamos 1, por que no
                                                                         # empieza desde 0), lo añadimos a la nueva lista
            group8RE.append(line[:b])    
            
    ##########################################################################################################################
    # Sabiendo ya los estados, vamos a sacar sus frames correspondientes
    ##########################################################################################################################

    it = 1 #Creamos un contador que nos indicara en que frame nos encontramos
    
    name_video = Dataset_rat[37:]
    
    cap = cv2.VideoCapture('/mnt/MD1200A/lconcha/videos/RAT_VIDEOS_CROPPED_RESIZE/'+ name_video +'.mp4') #archivo de video que vamos a leer

    np_frames = [] #lista en la que vamos a guardar los frames 

    while (cap.isOpened()): #leemos el video al cual vamos a crear su dataset

        ret, frame = cap.read()

        if ret == True:
            
            resize = cv2.resize(frame, (256,256), interpolation = cv2.INTER_AREA)
            gray= cv2.cvtColor(resize, cv2.COLOR_BGR2GRAY)
            for line in group8RE: #loope que recorre la lista con los grupos de 8
                if line[1:] == str(it): #si el numero coincide con el frame, lo añadimos a nuestra lista
                    np_frames.append(gray)
            it += 1

        if ret == False:
            break
            
    sets = [] #En esta lista vamos a separar en conjuntos de 8 los frames obtenidos anteriormente
    X_values = [] #En esta lista cada conjunto de 8 frames lo convertiremos en un array de shape [8 512 512]
    state = 'RE' #estado de todos los frames

    it = 0

    for i in range(0, len(np_frames), 8): #Recorremos np_frames, y los añadimos de 8 en 8 a sets
        sets.append(np_frames[i:i+8])

    for j in sets: #recorremos sets y concatenamos los 8 frames, para añadirlos a una nueva lista
        X = np.stack(j)
        X_values.append(X)

    with h5py.File('/mnt/MD1200A/lconcha/videos/256x256/Datasets_256_RE_32_blackwhite/'+ name_video + '_RE', 'w') as h5File: #Creamos en archivo .h5
        for line in X_values:
            it += 1
            Xset = h5File.create_dataset('X'+str(it), data= line)
            Yset = h5File.create_dataset('Y'+str(it), data = state)

In [8]:
def createDataset_EA_group32(Dataset_rat):

    ##########################################################################################################################
    # Ahora creamos un dataset para el estado de ataque, hacemos lo mismo pero con los frames de EA
    ##########################################################################################################################

    Y_list = [] #Lista en la que vamos a almacenar los valores de Y

    dataset = h5py.File(Dataset_rat,'r')

    for line in dataset:
        if line[0] == 'Y':
            Y_list.append(line)

    Y_list.sort(key = len)

    allEA = [] #Lista que almacenara todos los valores de Y con un estado EA
    count = 0 

    for i in Y_list:
        for j in dataset:

            if i == j: 
                if dataset[j].value == 'EA': 
                    count += 1
                    allEA.append(j + '/' +str(count))  

                    if count == 33: 
                        count = 0

                if dataset[j].value == 'RE':
                    count = 0

    sixfourEA = [] #en esta lista meteremos todos los valores solo si son consecutivos 32 veces.
    a = False

    for h in reversed(allEA): 
        b = h.find('/')

        if h[b:] == '/33': 
            a = True

        if a == True:
            sixfourEA.append(h)

        if h[b:] == '/1': #Cada vez que el numero llegue a 1 el estado del booleano cambiara a false y dejara de agregar a la
                          # lista datos, hasta que vuelva a encontrar un 33
            a = False

    group8EA = [] #De cada grupo de 64, cogeremos 8 (los que sean multiplos de 4) y los guardaremos en esta lista

    for line in reversed(sixfourEA): 
        b = line.find('/')
        if (line[b:] == '/1' or line[b:] == '/5' or line[b:] == '/9' or line[b:] == '/13' or line[b:] == '/17' 
        or line[b:] == '/21' or line[b:] == '/25' or line[b:] == '/29'): 
            group8EA.append(line[:b])

    ##########################################################################################################################
    # Sabiendo ya los estados, vamos a sacar sus frames correspondientes
    ##########################################################################################################################

    it = 1 

    name_video = Dataset_rat[37:] 

    cap = cv2.VideoCapture('/mnt/MD1200A/lconcha/videos/RAT_VIDEOS_CROPPED_RESIZE/'+ name_video +'.mp4') #archivo de video que vamos a leer 

    np_frames_EA = [] 

    while (cap.isOpened()): 

        ret, frame = cap.read()

        if ret == True:
            
            resize = cv2.resize(frame, (256,256), interpolation = cv2.INTER_AREA)
            gray= cv2.cvtColor(resize, cv2.COLOR_BGR2GRAY)
            for line in group8EA: 
                if line[1:] == str(it): 
                    np_frames_EA.append(gray)
            it += 1

        if ret == False:
            break

    sets = [] 
    X_values = [] 
    state = 'EA' 

    it = 0

    for i in range(0, len(np_frames_EA), 8): 
        sets.append(np_frames_EA[i:i+8])

    for j in sets: 
        X = np.stack(j)
        X_values.append(X)

    with h5py.File('/mnt/MD1200A/lconcha/videos/256x256/Datasets_256_EA_32_blackwhite/'+ name_video + '_EA', 'w') as h5File: #Creamos en archivo .h5
        for line in X_values:
            it += 1
            Xset = h5File.create_dataset('X'+str(it), data= line)
            Yset = h5File.create_dataset('Y'+str(it), data = state)

In [6]:
#Le decimos la ruta 
ruta = r'/mnt/MD1200A/lconcha/videos/Datasets'

#hacemos una lista de todos los archivos de la ruta
archivo = os.listdir(ruta)

#un loop para que ejecute el programa anterior a todos los elementos de la lista
for i in archivo:
    createDataset_EA_group32('/mnt/MD1200A/lconcha/videos/Datasets/' + i)
    print (i + ' '+ 'completed_EA')
    
    createDataset_RE_group32('/mnt/MD1200A/lconcha/videos/Datasets/' + i)
    print (i + ' '+ 'completed_RE')

/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:31: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


OSError: Unable to create file (unable to open file: name = '/mnt/MD1200A/lconcha/videos/128x128_Datasets_corregidos/Datasets_EA_32_corregidos/fernando/1.2018-1-9.Rat8_EA', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [ ]:
#Le decimos la ruta 
ruta1 = r'/mnt/MD1200A/lconcha/videos/Datasets'
ruta2 = r'/mnt/MD1200A/lconcha/videos/256x256/Datasets_256_EA_32_blackwhite'

#hacemos una lista de todos los archivos de la ruta
archivo1 = os.listdir(ruta1)
archivo2 = os.listdir(ruta2)
archivo3 = []

for l in archivo2:
    name = l.find('_')
    archivo3.append(l[:name])
       
        

lista = list(set(archivo1)-set(archivo3))


for i in lista:
    createDataset_EA_group32('/mnt/MD1200A/lconcha/videos/Datasets/' + i)
    print (i + ' '+ 'completed_EA')
    
    createDataset_RE_group32('/mnt/MD1200A/lconcha/videos/Datasets/' + i)
    print (i + ' '+ 'completed_RE')

/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:31: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


90.2018-2-14.Rat25 completed_EA


/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:32: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


90.2018-2-14.Rat25 completed_RE
88.2018-2-14.Rat30 completed_EA
88.2018-2-14.Rat30 completed_RE
.ipynb_checkpoint completed_EA
.ipynb_checkpoint completed_RE
89.2018-2-14.Rat25 completed_EA
89.2018-2-14.Rat25 completed_RE
89.2018-2-14.Rat26 completed_EA
89.2018-2-14.Rat26 completed_RE
89.2018-2-14.Rat30 completed_EA
89.2018-2-14.Rat30 completed_RE
90.2018-2-14.Rat26 completed_EA


# Grupos de 32 frames, pero con los Datasets editados por mi

In [2]:
def createDataset_RE_group32(Dataset_rat):
    
    Y_list = [] #Lista en la que vamos a almacenar los valores de Y

    dataset = h5py.File(Dataset_rat,'r')


    #Recorremos el dataset, si la posicion 0 es una Y, añadimos el valor a nuestra lista
    for line in dataset:
        if line[0] == 'Y':
            Y_list.append(line)


    Y_list.sort(key = len) #Nos ordena la lista. Y va acompañado de un valor numerico pero al estar en forma de string, 
                           #tenemos que ordenarlos en funcion de su longitud

    allRE = [] #Lista que almacenara todos los valores de Y con un estado RE
    count = 0 #Contador para comprobar si son 32 frames consecutivos con el mismo estado RE

    for i in Y_list:  #Recorremos la lista y las lineas del dataset
        for j in dataset:

            if i == j: #Si coinciden los valores, comprobamos que el estado de ese valor sea RE
                if dataset[j].value == 'RE': #si el estado es RE lo añadimos a la lista y sumamos 1 al contador
                    count += 1
                    allRE.append(j + '/' +str(count)) #ademas tambien le vamos a añadir al nombre el contador para poder
                                                      #analizar posteriormente si son consecutivos 

                    if count == 33: #cuando el contador llegue a 33 (empieza en 1), se volvera a poner a 0
                        count = 0

                if dataset[j].value == 'EA': #si encuentra un valor de EA para Y, se pondra a 0 el contador
                    count = 0
                    
    sixfourRE = [] #en esta lista meteremos todos los valores solo si son consecutivos 32 veces.
    a = False

    for h in reversed(allRE): #leemos la lista de forma inversa, para comprobar que son consecutivos haremos una cuenta atras
                              # si empieza en 33 empezara a añadir valores a la lista
        b = h.find('/')

        if h[b:] == '/33': 
            a = True

        if a == True:
            sixfourRE.append(h)

        if h[b:] == '/1': #Cada vez que el numero llegue a 1 el estado del booleano cambiara a false y dejara de agregar a la
                          # lista datos, hasta que vuelva a encontrar un 33
            a = False    
            
            
    group8RE = [] #De cada grupo de 32, cogeremos 8 (los que sean multiplos de 4) y los guardaremos en esta lista

    for line in reversed(sixfourRE): #recorremos la lista anterior invertida (la volvemos a poner en orden ascendente)
        b = line.find('/')
        if (line[b:] == '/1' or line[b:] == '/5' or line[b:] == '/9' or line[b:] == '/13' or line[b:] == '/17' 
        or line[b:] == '/21' or line[b:] == '/25' or line[b:] == '/29'): # si es multiplo de 8 (le sumamos 1, por que no
                                                                         # empieza desde 0), lo añadimos a la nueva lista
            group8RE.append(line[:b])    
            
    ##########################################################################################################################
    # Sabiendo ya los estados, vamos a sacar sus frames correspondientes
    ##########################################################################################################################

    it = 1 #Creamos un contador que nos indicara en que frame nos encontramos
    
    name_video = Dataset_rat[46:]
    
    cap = cv2.VideoCapture('/mnt/MD1200A/lconcha/videos/RAT_VIDEOS_CROPPED_RESIZE/'+ name_video +'.mp4') #archivo de video que vamos a leer

    np_frames = [] #lista en la que vamos a guardar los frames 

    while (cap.isOpened()): #leemos el video al cual vamos a crear su dataset

        ret, frame = cap.read()

        if ret == True:
            
            resize = cv2.resize(frame, (256,256), interpolation = cv2.INTER_AREA)
            for line in group8RE: #loope que recorre la lista con los grupos de 8
                if line[1:] == str(it): #si el numero coincide con el frame, lo añadimos a nuestra lista
                    np_frames.append(resize)
            it += 1

        if ret == False:
            break
            
    sets = [] #En esta lista vamos a separar en conjuntos de 8 los frames obtenidos anteriormente
    X_values = [] #En esta lista cada conjunto de 8 frames lo convertiremos en un array de shape [8 512 512]
    state = 'RE' #estado de todos los frames

    it = 0

    for i in range(0, len(np_frames), 8): #Recorremos np_frames, y los añadimos de 8 en 8 a sets
        sets.append(np_frames[i:i+8])

    for j in sets: #recorremos sets y concatenamos los 8 frames, para añadirlos a una nueva lista
        X = np.stack(j)
        X_values.append(X)

    with h5py.File('/mnt/MD1200A/lconcha/videos/256x256_Datasets_corregidos/Datasets_256_RE_corregidos/'+ name_video + '_RE', 'w') as h5File: #Creamos en archivo .h5
        for line in X_values:
            it += 1
            Xset = h5File.create_dataset('X'+str(it), data= line)
            Yset = h5File.create_dataset('Y'+str(it), data = state)

In [3]:
def createDataset_EA_group32(Dataset_rat):

    ##########################################################################################################################
    # Ahora creamos un dataset para el estado de ataque, hacemos lo mismo pero con los frames de EA
    ##########################################################################################################################

    Y_list = [] #Lista en la que vamos a almacenar los valores de Y

    dataset = h5py.File(Dataset_rat,'r')

    for line in dataset:
        if line[0] == 'Y':
            Y_list.append(line)

    Y_list.sort(key = len)

    allEA = [] #Lista que almacenara todos los valores de Y con un estado EA
    count = 0 

    for i in Y_list:
        for j in dataset:

            if i == j: 
                if dataset[j].value == 'EA': 
                    count += 1
                    allEA.append(j + '/' +str(count))  

                    if count == 33: 
                        count = 0

                if dataset[j].value == 'RE':
                    count = 0

    sixfourEA = [] #en esta lista meteremos todos los valores solo si son consecutivos 32 veces.
    a = False

    for h in reversed(allEA): 
        b = h.find('/')

        if h[b:] == '/33': 
            a = True

        if a == True:
            sixfourEA.append(h)

        if h[b:] == '/1': #Cada vez que el numero llegue a 1 el estado del booleano cambiara a false y dejara de agregar a la
                          # lista datos, hasta que vuelva a encontrar un 33
            a = False

    group8EA = [] #De cada grupo de 64, cogeremos 8 (los que sean multiplos de 4) y los guardaremos en esta lista

    for line in reversed(sixfourEA): 
        b = line.find('/')
        if (line[b:] == '/1' or line[b:] == '/5' or line[b:] == '/9' or line[b:] == '/13' or line[b:] == '/17' 
        or line[b:] == '/21' or line[b:] == '/25' or line[b:] == '/29'): 
            group8EA.append(line[:b])

    ##########################################################################################################################
    # Sabiendo ya los estados, vamos a sacar sus frames correspondientes
    ##########################################################################################################################

    it = 1 

    name_video = Dataset_rat[46:] 

    cap = cv2.VideoCapture('/mnt/MD1200A/lconcha/videos/RAT_VIDEOS_CROPPED_RESIZE/'+ name_video +'.mp4') #archivo de video que vamos a leer 

    np_frames_EA = [] 

    while (cap.isOpened()): 

        ret, frame = cap.read()

        if ret == True:
            
            resize = cv2.resize(frame, (256,256), interpolation = cv2.INTER_AREA)
            for line in group8EA: 
                if line[1:] == str(it): 
                    np_frames_EA.append(resize)
            it += 1

        if ret == False:
            break

    sets = [] 
    X_values = [] 
    state = 'EA' 

    it = 0

    for i in range(0, len(np_frames_EA), 8): 
        sets.append(np_frames_EA[i:i+8])

    for j in sets: 
        X = np.stack(j)
        X_values.append(X)

    with h5py.File('/mnt/MD1200A/lconcha/videos/256x256_Datasets_corregidos/Datasets_256_EA_corregidos/'+ name_video + '_EA', 'w') as h5File: #Creamos en archivo .h5
        for line in X_values:
            it += 1
            Xset = h5File.create_dataset('X'+str(it), data= line)
            Yset = h5File.create_dataset('Y'+str(it), data = state)

In [ ]:
#Le decimos la ruta 
ruta = r'/mnt/MD1200A/lconcha/videos/Datasets_fernando'

#hacemos una lista de todos los archivos de la ruta
archivo = os.listdir(ruta)

#un loop para que ejecute el programa anterior a todos los elementos de la lista
for i in archivo:
    createDataset_EA_group32('/mnt/MD1200A/lconcha/videos/Datasets_fernando/' + i)
    print (i + ' '+ 'completed_EA')
    
    createDataset_RE_group32('/mnt/MD1200A/lconcha/videos/Datasets_fernando/' + i)
    print (i + ' '+ 'completed_RE')

/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:24: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:31: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


1.2018-1-9.Rat8 completed_EA


/cm/shared/apps/anaconda2/4.3.1/envs/jupyter3.7/lib/python3.7/site-packages/ipykernel_launcher.py:32: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


1.2018-1-9.Rat8 completed_RE
1.2018-1-9.Rat9 completed_EA
1.2018-1-9.Rat9 completed_RE
10.2017-11-10.Rat11 completed_EA
10.2017-11-10.Rat11 completed_RE
10.2017-11-10.Rat12 completed_EA
10.2017-11-10.Rat12 completed_RE
11.2017-11-13.Rat11 completed_EA
11.2017-11-13.Rat11 completed_RE
11.2017-11-13.Rat12 completed_EA
11.2017-11-13.Rat12 completed_RE
12.2017-12-13.Rat13 completed_EA
12.2017-12-13.Rat13 completed_RE
12.2017-12-13.Rat18 completed_EA
12.2017-12-13.Rat18 completed_RE
13.2017-12-14.Rat13 completed_EA
13.2017-12-14.Rat13 completed_RE
13.2017-12-14.Rat18 completed_EA
13.2017-12-14.Rat18 completed_RE
14.2017-12-14.Rat13 completed_EA
14.2017-12-14.Rat13 completed_RE
14.2017-12-14.Rat18 completed_EA
14.2017-12-14.Rat18 completed_RE
15.2017-12-15.Rat13 completed_EA
15.2017-12-15.Rat13 completed_RE
15.2017-12-15.Rat18 completed_EA
15.2017-12-15.Rat18 completed_RE
16.2018-1-19.Rat13 completed_EA
16.2018-1-19.Rat13 completed_RE
16.2018-1-19.Rat18 completed_EA
16.2018-1-19.Rat18 comple